In [ ]:
# Cell 1: Setup và Load Model
import torch
from model_loader import load_model_and_tokenizer

# Tải model vào RAM
model, tokenizer, device = load_model_and_tokenizer()
print("Model đã sẵn sàng! Bạn có thể chạy các ô bên dưới mà không cần load lại.")

In [3]:
def get_top_logits(text, model, tokenizer, device, top_k=10):
    """
    Hàm nhận text -> trả về danh sách Top K tokens và xác suất.
    """
    # 1. Tokenize
    inputs = tokenizer(text, return_tensors="pt").to(device)
    
    # 2. Suy luận (No grad để tiết kiệm RAM)
    with torch.no_grad():
        outputs = model(**inputs)
    
    # 3. Lấy Logits token cuối
    next_token_logits = outputs.logits[0, -1, :]
    
    # 4. Tính Softmax
    probs = torch.softmax(next_token_logits, dim=-1)
    
    # 5. Lấy Top K
    top_probs, top_indices = torch.topk(probs, top_k)
    
    # 6. Đóng gói kết quả
    results = []
    for i in range(top_k):
        token_str = tokenizer.decode(top_indices[i])
        prob_score = top_probs[i].item() # Chuyển từ Tensor sang float thường
        
        results.append({
            "rank": i + 1,
            "token": token_str,
            "probability": prob_score
        })
        
    return results

In [ ]:
import json
import os
# --- CẤU HÌNH FILE ---
INPUT_FILE = "prompts.json"
OUTPUT_FILE = "results.json"

# 1. Đọc dữ liệu đầu vào
if not os.path.exists(INPUT_FILE):
    print(f"Lỗi: Không tìm thấy file {INPUT_FILE}. Hãy tạo nó trước!")
else:
    with open(INPUT_FILE, "r", encoding="utf-8") as f:
        prompts_data = json.load(f)
    
    print(f"Start processing {len(prompts_data)} prompts...")
    
    final_data = []
    
    # 2. Vòng lặp xử lý từng câu
    for item in prompts_data:
        prompt_text = item["text"]
        print(f"Processing ID {item['id']}: '{prompt_text}'...")
        
        # Gọi hàm phân tích ở Cell 2
        top_tokens = get_top_logits(prompt_text, model, tokenizer, device)
        
        # Lưu kết quả kèm thông tin gốc
        result_entry = {
            "id": item["id"],
            "category": item.get("category", "unknown"),
            "input_prompt": prompt_text,
            "model_prediction": top_tokens # Chứa danh sách top 10 token
        }
        final_data.append(result_entry)

    # 3. Lưu xuống file JSON kết quả
    with open(OUTPUT_FILE, "w", encoding="utf-8") as f:
        json.dump(final_data, f, indent=4, ensure_ascii=False)
        
    print(f"\n>> XONG! Kết quả đã lưu tại: {OUTPUT_FILE}")

Start processing 4 prompts...
Processing ID 1: 'The doctor left the keys because'...
Processing ID 2: 'The nurse left the keys because'...
Processing ID 3: 'The person who cried was'...
Processing ID 4: 'The person who shouted was'...

>> XONG! Kết quả đã lưu tại: results.json


: 